# Coding Project Analysis JEM Forms


### Data Output: Date, Specimen ID, Rig Opertator, Layer, Cell Type Prediction (Human)

In [1]:
import os
import csv
import json
import fnmatch
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.options.display.max_columns = None  #Displays all the columns 

In [3]:
#Still have to fix this and add more definitions for common functions

def gen_fil(df, col, cond):
    '''
    Filters the column with a certain condition
    df: dataframe
    col: column
    cond: condition
    '''
    filtered = df[df[col] == cond]
    return filtered

def gen_cat(df, col):
    '''
    Assigns astype to the column
    df: dataframe
    col: column
    '''    
    category = df[col].astype('category')
    return category

def cell_count(P_num):
    '''
    Counts number of Lims tube id based on P# as number of cells
    df = dataframe
    count() = counts number
    '''
    user = df1[df1['Lims tube id'].str.contains(P_num)]
    return user['Lims tube id'].count()

def P_user(P_num):
    '''
    Prints cell_count 
    '''
    print("Total cells:", cell_count(P_num))

In [4]:
#Post patch class Script by Rusty

def nucleated(x):
    nuc_high_seal = x[(x['extraction.postPatch'] == 'Nucleated') | 
                      (((x['extraction.postPatch'] == 'nucleus_visible') | 
                      (x['extraction.postPatch'] == 'nucleus_present')) & 
                      (x['extraction.endPipetteR'] >= 500))]
    return nuc_high_seal

def partial_nucleated(y):
    nuc_low_seal = y[(y['extraction.postPatch'] == 'Partial-Nucleus') | 
                     (((y['extraction.postPatch'] == 'nucleus_present') | 
                     (y['extraction.postPatch'] == 'nucleus_visible')) & 
                     (y['extraction.endPipetteR'] <= 499))]
    return nuc_low_seal

def outside_out(z):
    no_high_seal = z[(z['extraction.postPatch'] == 'Outside-Out') | 
                     (((z['extraction.postPatch'] == 'nucleus_absent') | 
                     (z['extraction.postPatch'] == 'no_nucleus_visible')) & 
                     (z['extraction.endPipetteR'] >= 500))]
    return no_high_seal

def no_seal(w): 
    no_low_seal = w[(w['extraction.postPatch'] == 'No-Seal') | 
                    (((w['extraction.postPatch'] == 'nucleus_absent') | 
                    (w['extraction.postPatch'] == 'no_nucleus_visible')) & 
                    (w['extraction.endPipetteR'] <= 499))]
    return no_low_seal
    
def entire_cell(v):
    entire = v[(v['extraction.postPatch'] == 'Entire-Cell') | 
               (v['extraction.postPatch'] == 'entire_cell')]
    return entire

#variable['post_patch'] = 'Term'
#Term is an output displayed in the Post_Patch column

def reclassify(df):
    nu = nucleated(df)
    nu['post_patch'] = 'Nuc-high seal' 
    oo = outside_out(df)
    oo['post_patch'] = 'No-high seal'
    pn = partial_nucleated(df)
    pn['post_patch'] = 'Nuc-low seal'
    ns = no_seal(df)
    ns['post_patch'] = 'No-low seal'
    ec = entire_cell(df)
    ec['post_patch'] = 'Entire cell'
    return  nu, oo, pn, ns, ec

def concat_df(a, b, c, d, e):
    frames = (a, b, c, d, e)
    df = pd.concat(frames)
    return df

def postpatch_reclass(df):
    return concat_df(*reclassify(df))

#json_df = postpatch_reclass(json_df)

In [5]:
def get_jsons(dirname, expt):
    json_paths = []
    for jfile in os.listdir(dirname):
        if fnmatch.fnmatch(jfile, '*.%s.json' %expt):
            jpath = os.path.join(dirname, jfile)
            json_paths.append(jpath)
    return json_paths

In [6]:
#make list of json paths for every json file in MIES Experiments directory on 279
json_list = get_jsons("//allen/programs/celltypes/workgroups/279/Patch-Seq/all-metadata-files/", "PS")
json_list

['//allen/programs/celltypes/workgroups/279/Patch-Seq/all-metadata-files/Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176.11.06.PS.json',
 '//allen/programs/celltypes/workgroups/279/Patch-Seq/all-metadata-files/Rbp4-Cre_KL100;Ai14-357010.03.01.PS.json',
 '//allen/programs/celltypes/workgroups/279/Patch-Seq/all-metadata-files/Oxtr-T2A-Cre;Pvalb-T2A-FlpO;Ai65-370683.03.01.PS.json',
 '//allen/programs/celltypes/workgroups/279/Patch-Seq/all-metadata-files/Ntsr1-Cre_GN220;Ai14-361815.05.01.PS.json',
 '//allen/programs/celltypes/workgroups/279/Patch-Seq/all-metadata-files/Gad2-IRES-Cre;Ai14-296621.05.02.PS.json',
 '//allen/programs/celltypes/workgroups/279/Patch-Seq/all-metadata-files/Cux2-CreERT2;Ai14-315357.03.02.PS.json',
 '//allen/programs/celltypes/workgroups/279/Patch-Seq/all-metadata-files/Rbp4-Cre_KL100;Ai14-280355.05.01.PS.json',
 '//allen/programs/celltypes/workgroups/279/Patch-Seq/all-metadata-files/H17.06.015.13.08.PS.json',
 '//allen/programs/celltypes/workgroups/279/Patch-Seq/all

In [7]:
def flatten_attempts(slice_info, form_version):
    """Return flattened slice metadata dataframe.
    
    Parameters
    ----------
    slice_info : dict
        A dictionary of slices with nested pipette attempts.
    
    form_version : string
        A string containing the JEM form version.
        (Pre-version 2 contains IVSCC, PatchSeq and Electroporation arrays)
    Returns
    -------
    attempts_slice_df : pandas dataframe
        A dataframe of all pipette attempts along with all slice metadata.
    """
    
    df = json_normalize(slice_info)
    if form_version >= "2":
        ps_array_name = "pipettes"
    else:
        ps_array_name = "pipettesPatchSeqPilot"
    try:
        attempts_df = json_normalize(slice_info[ps_array_name])
        attempts_df["limsSpecName"] = df["limsSpecName"].values[0]
        attempts_df["attempt"] = [p+1 for p in attempts_df.index.values]
        attempts_slice_df = pd.merge(df, attempts_df, how="outer", on="limsSpecName")
        attempts_slice_df.drop(ps_array_name, axis=1, inplace=True)
        return attempts_slice_df
    except IndexError:
        pass

def is_field(df, colname):
    """Determine whether a column name exists in a dataframe.
    
    Parameters
    ----------
    df : a Pandas dataframe
    colname : string
        
    Returns
    -------
    Boolean
        Boolean value indicating if the colname exists in the dataframe.
    """
    
    try:
        df[colname]
        return True
    except KeyError:
        return False

### json_df

In [8]:
json_df = pd.DataFrame()

for json_path in json_list:
    with open(json_path) as data_file:
        slice_info = json.load(data_file)
        if is_field(slice_info, "formVersion"):
            jem_version = slice_info["formVersion"]
        else:
            jem_version = "1.0.0"
        flat_df = flatten_attempts(slice_info, jem_version)
        json_df = pd.concat([json_df, flat_df], axis=0)
        
json_df.head()

,acsfProductionDate,approach.anatomicalLocation,approach.autoRoi,approach.cellHealth,approach.corticalLayer,approach.creCell,approach.depth,approach.detailedLocation,approach.manualRoi,approach.otherPilotName,approach.pilotName,approach.pilotTest01,approach.pilotTest03,approach.pilotTest04,approach.pilotTest05,approach.sliceHealth,attempt,autoRoi,badSweeps,blankFillDate,date,depth,experimentType,extraction.bfiWitness,extraction.endPipetteR,extraction.extractionNotes,extraction.extractionObservations,extraction.nucleus,extraction.postPatch,extraction.pressureApplied,extraction.retractionPressureApplied,extraction.sampleObservations,extraction.timeEnd,extraction.timeExtractionEnd,extraction.timeExtractionStart,extraction.timeRetractionEnd,extraction.timeRetractionStart,extraction.tubeID,failureCause,failureNotes,flipped,formVersion,freeFailureNotes,internalFillDate,internalSolution.concentrationAlexa,internalSolution.concentrationBiocytin,internalSolution.concentrationRnaseInhibitor,internalSolution.version,internalSolution.volume,limsSpecName,manualRoi,pipetteSpecName,qcNotes,recording.accessR,recording.humanCellTypePrediction,recording.membraneV,recording.pipetteR,recording.rheobase,recording.timeStart,recording.timeWholeCellStart,rigNumber,rigOperator,sliceNotes,sliceQuality,status,successNotes,wellID
0,2018-04-18,NaN,NaN,3,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,1.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,no,nucleus_absent,-80.0,-80.0,NaN,NaN,13:20:14 -07:00,13:20:13 -07:00,13:20:14 -07:00,NaN,PXS4_180418_551_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:19:59 -07:00,13:20:00 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN
1,2018-04-18,NaN,NaN,2,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,2.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,NaN,nucleus_present,-80.0,-80.0,NaN,NaN,13:21:44 -07:00,13:21:44 -07:00,13:21:45 -07:00,NaN,PXS4_180418_552_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:21:26 -07:00,13:21:31 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN
2,2018-04-18,NaN,NaN,2,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,3.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,no,nucleus_absent,-80.0,-80.0,NaN,NaN,13:22:39 -07:00,13:22:38 -07:00,13:22:39 -07:00,NaN,PXS4_180418_553_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:22:25 -07:00,13:22:27 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN
3,2018-04-18,NaN,NaN,2,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,4.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,not_intentionally,nucleus_present,-80.0,-80.0,NaN,NaN,13:23:59 -07:00,13:23:58 -07:00,13:23:59 -07:00,NaN,PXS4_180418_554_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:23:04 -07:00,13:23:09 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN
4,2018-04-18,NaN,NaN,2,NaN,Cre+,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2,5.0,None,NaN,2018-03-22,2018-04-18 13:17:02 -07:00,-50,NaN,NaN,1000,NaN,NaN,NaN,nucleus_absent,-80.0,-80.0,NaN,NaN,13:25:01 -07:00,13:25:01 -07:00,13:25:01 -07:00,NaN,PXS4_180418_555_A01,NaN,NaN,No,2.0.3,NaN,2018-03-29,NaN,NaN,NaN,NaN,NaN,Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176...,VISp5,NaN,NaN,NaN,NaN,NaN,6,NaN,13:24:37 -07:00,13:24:40 -07:00,MP2,pashad,NaN,Damaged,SUCCESS,NaN,NaN


### Date column & include post_patch column

In [9]:
json_df['date'] = json_df['date'].str[:10] #Strip away the time
json_df['date'] = pd.to_datetime(json_df['date']) #Converting to YYYY-MM-DD
json_df.set_index('date', inplace = True)

In [10]:
json_df = postpatch_reclass(json_df) #Makes post patch column

### Creating new dataframe based on date start selection

In [11]:
#['Start date' : 'End date'] & Production Start Date = '2017-10-01' 
start_date = input('Enter the start date for your dataframe (YYYY-MM-DD): ')
json_df1 = json_df[start_date:]

json_df1.sort_index(inplace = True)
json_df1 = gen_fil(json_df1, 'status', 'SUCCESS') 

Enter the start date for your dataframe (YYYY-MM-DD): 2017-10-01


In [12]:
json_df1.head()

,acsfProductionDate,approach.anatomicalLocation,approach.autoRoi,approach.cellHealth,approach.corticalLayer,approach.creCell,approach.depth,approach.detailedLocation,approach.manualRoi,approach.otherPilotName,approach.pilotName,approach.pilotTest01,approach.pilotTest03,approach.pilotTest04,approach.pilotTest05,approach.sliceHealth,attempt,autoRoi,badSweeps,blankFillDate,depth,experimentType,extraction.bfiWitness,extraction.endPipetteR,extraction.extractionNotes,extraction.extractionObservations,extraction.nucleus,extraction.postPatch,extraction.pressureApplied,extraction.retractionPressureApplied,extraction.sampleObservations,extraction.timeEnd,extraction.timeExtractionEnd,extraction.timeExtractionStart,extraction.timeRetractionEnd,extraction.timeRetractionStart,extraction.tubeID,failureCause,failureNotes,flipped,formVersion,freeFailureNotes,internalFillDate,internalSolution.concentrationAlexa,internalSolution.concentrationBiocytin,internalSolution.concentrationRnaseInhibitor,internalSolution.version,internalSolution.volume,limsSpecName,manualRoi,pipetteSpecName,qcNotes,recording.accessR,recording.humanCellTypePrediction,recording.membraneV,recording.pipetteR,recording.rheobase,recording.timeStart,recording.timeWholeCellStart,rigNumber,rigOperator,sliceNotes,sliceQuality,status,successNotes,wellID,post_patch
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,Nucleated Patch - Retraction Pressure,NaN,NaN,NaN,NaN,3,4.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,1000,NaN,Fluorescence in Pipette,no,nucleus_present,-32.0,-32.0,No Bubbles,NaN,15:43:26,15:42:58,15:45:24,15:43:27,055,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Chrna2-Cre_OE25;Ai14-351067.04.01,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,15:37:03,15:38:40,5,rustym,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B03_M,Nuc-high seal
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,3,1.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,3000,1.5 min retraction time. Can see alexa fill po...,"Fluorescence in Pipette,Cell Dimmed,Cell Shrunk",no,nucleus_present,-30.0,-30.0,No Bubbles,NaN,15:24:14,15:22:54,15:25:49,15:24:15,356,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Chrna2-Cre_OE25;Ai14-351067.03.02,NaN,NaN,NaN,NaN,NaN,NaN,5.3,NaN,15:14:50,15:16:11,7,lindsayn,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B02_M,Nuc-high seal
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 6a",NaN,None,NaN,NaN,NaN,NaN,2,4.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,16.2,NaN,NaN,no,nucleus_absent,-30.0,-30.0,No Bubbles,NaN,15:56:38,15:56:04,15:58:43,15:56:39,406,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Gad2-IRES-Cre;Ai14-350672.04.01,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,15:50:28,15:51:06,6,lisak,NaN,NaN,SUCCESS,Access Resistance Increased,EPhys_171002_01_B01_M,No-high seal
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 4",NaN,Nucleated Patch - Retraction Pressure,NaN,NaN,NaN,NaN,3,1.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,10,NaN,Fluorescence in Pipette,no,nucleus_absent,-32.5,-32.5,"Large Bubbles,Solution in Pipette Shank",NaN,14:48:35,14:47:57,14:51:55,14:48:36,053,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Chrna2-Cre_OE25;Ai14-351067.04.01,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,14:41:05,14:42:42,5,rustym,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B03_M,No-high seal
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,2,2.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,30,No alexa fill post experiment,"Fluorescence in Pipette,Cell Dimmed,Cell Shrunk",no,nucleus_absent,-30.0,-30.0,No Bubbles,NaN,15:56:10,15:55:26,16:01:35,15:56:11,357,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Chrna2-Cre_OE25;Ai14-351067.03.02,NaN,NaN,NaN,NaN,NaN,NaN,5.5,NaN,15:46:38,15:47:49,7,lindsayn,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B02_M,No-high seal


### Create a filter for Human & Mouse dataframe

In [14]:
json_df1['human?'] = json_df1['limsSpecName'].str.match(r"H\d\d") #Creates new boolean column

json_df2 = gen_fil(json_df1, 'human?', True) #HUMAN
json_df3 = gen_fil(json_df1, 'human?', False) #MOUSE
json_df1.head()

#json_df2 = json_df1[json_df1['limsSpecName'].str.match('H1')] #Human
#json_df3 = json_df1[json_df1['limsSpecName'] != 'H'] #Mouse

,acsfProductionDate,approach.anatomicalLocation,approach.autoRoi,approach.cellHealth,approach.corticalLayer,approach.creCell,approach.depth,approach.detailedLocation,approach.manualRoi,approach.otherPilotName,approach.pilotName,approach.pilotTest01,approach.pilotTest03,approach.pilotTest04,approach.pilotTest05,approach.sliceHealth,attempt,autoRoi,badSweeps,blankFillDate,depth,experimentType,extraction.bfiWitness,extraction.endPipetteR,extraction.extractionNotes,extraction.extractionObservations,extraction.nucleus,extraction.postPatch,extraction.pressureApplied,extraction.retractionPressureApplied,extraction.sampleObservations,extraction.timeEnd,extraction.timeExtractionEnd,extraction.timeExtractionStart,extraction.timeRetractionEnd,extraction.timeRetractionStart,extraction.tubeID,failureCause,failureNotes,flipped,formVersion,freeFailureNotes,internalFillDate,internalSolution.concentrationAlexa,internalSolution.concentrationBiocytin,internalSolution.concentrationRnaseInhibitor,internalSolution.version,internalSolution.volume,limsSpecName,manualRoi,pipetteSpecName,qcNotes,recording.accessR,recording.humanCellTypePrediction,recording.membraneV,recording.pipetteR,recording.rheobase,recording.timeStart,recording.timeWholeCellStart,rigNumber,rigOperator,sliceNotes,sliceQuality,status,successNotes,wellID,post_patch,human?
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,Nucleated Patch - Retraction Pressure,NaN,NaN,NaN,NaN,3,4.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,1000,NaN,Fluorescence in Pipette,no,nucleus_present,-32.0,-32.0,No Bubbles,NaN,15:43:26,15:42:58,15:45:24,15:43:27,055,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Chrna2-Cre_OE25;Ai14-351067.04.01,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,15:37:03,15:38:40,5,rustym,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B03_M,Nuc-high seal,False
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,3,1.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,3000,1.5 min retraction time. Can see alexa fill po...,"Fluorescence in Pipette,Cell Dimmed,Cell Shrunk",no,nucleus_present,-30.0,-30.0,No Bubbles,NaN,15:24:14,15:22:54,15:25:49,15:24:15,356,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Chrna2-Cre_OE25;Ai14-351067.03.02,NaN,NaN,NaN,NaN,NaN,NaN,5.3,NaN,15:14:50,15:16:11,7,lindsayn,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B02_M,Nuc-high seal,False
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 6a",NaN,None,NaN,NaN,NaN,NaN,2,4.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,16.2,NaN,NaN,no,nucleus_absent,-30.0,-30.0,No Bubbles,NaN,15:56:38,15:56:04,15:58:43,15:56:39,406,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Gad2-IRES-Cre;Ai14-350672.04.01,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,15:50:28,15:51:06,6,lisak,NaN,NaN,SUCCESS,Access Resistance Increased,EPhys_171002_01_B01_M,No-high seal,False
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 4",NaN,Nucleated Patch - Retraction Pressure,NaN,NaN,NaN,NaN,3,1.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,10,NaN,Fluorescence in Pipette,no,nucleus_absent,-32.5,-32.5,"Large Bubbles,Solution in Pipette Shank",NaN,14:48:35,14:47:57,14:51:55,14:48:36,053,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Chrna2-Cre_OE25;Ai14-351067.04.01,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,14:41:05,14:42:42,5,rustym,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B03_M,No-high seal,False
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,2,2.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,30,No alexa fill post experiment,"Fluorescence in Pipette,Cell Dimmed,Cell Shrunk",no,nucleus_absent,-30.0,-30.0,No Bubbles,NaN,15:56:10,15:55:26,16:01:35,15:56:11,357,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Chrna2-Cre_OE25;Ai14-351067.03.02,NaN,NaN,NaN,NaN,NaN,NaN,5.5,NaN,15:46:38,15:47:49,7,lindsayn,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B02_M,No-high seal,False


In [15]:
#Human
json_df2.head()

,acsfProductionDate,approach.anatomicalLocation,approach.autoRoi,approach.cellHealth,approach.corticalLayer,approach.creCell,approach.depth,approach.detailedLocation,approach.manualRoi,approach.otherPilotName,approach.pilotName,approach.pilotTest01,approach.pilotTest03,approach.pilotTest04,approach.pilotTest05,approach.sliceHealth,attempt,autoRoi,badSweeps,blankFillDate,depth,experimentType,extraction.bfiWitness,extraction.endPipetteR,extraction.extractionNotes,extraction.extractionObservations,extraction.nucleus,extraction.postPatch,extraction.pressureApplied,extraction.retractionPressureApplied,extraction.sampleObservations,extraction.timeEnd,extraction.timeExtractionEnd,extraction.timeExtractionStart,extraction.timeRetractionEnd,extraction.timeRetractionStart,extraction.tubeID,failureCause,failureNotes,flipped,formVersion,freeFailureNotes,internalFillDate,internalSolution.concentrationAlexa,internalSolution.concentrationBiocytin,internalSolution.concentrationRnaseInhibitor,internalSolution.version,internalSolution.volume,limsSpecName,manualRoi,pipetteSpecName,qcNotes,recording.accessR,recording.humanCellTypePrediction,recording.membraneV,recording.pipetteR,recording.rheobase,recording.timeStart,recording.timeWholeCellStart,rigNumber,rigOperator,sliceNotes,sliceQuality,status,successNotes,wellID,post_patch,human?
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-10-10,10/09/2017,NaN,None,NaN,NaN,None,NaN,NaN,"TCx, layer 3",NaN,None,NaN,NaN,NaN,NaN,3,7.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,10,alexa fill visible post experiment.,NaN,no,nucleus_absent,-30.0,-50.0,No Bubbles,NaN,18:29:42,18:29:04,18:35:07,18:29:42,056,NaN,NaN,No,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,H17.03.014.11.10,NaN,NaN,NaN,NaN,NaN,NaN,4.8,NaN,18:19:05,18:20:42,7,rustym,NaN,NaN,SUCCESS,NaN,EPhys_171009_03_A03_H,No-high seal,True
2017-10-10,10/09/2017,NaN,None,NaN,NaN,None,NaN,NaN,"TCx, layer 3",NaN,None,NaN,NaN,NaN,NaN,2,6.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,10,alexa fill visible post experiment.\n,NaN,no,nucleus_absent,-80.0,-80.0,Medium Bubbles,NaN,21:17:16,21:16:36,21:20:36,21:17:18,060,NaN,NaN,No,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,H17.03.014.11.10,NaN,NaN,NaN,NaN,Pyramidal,NaN,3.4,NaN,21:04:54,21:06:49,7,rustym,NaN,NaN,SUCCESS,NaN,EPhys_171009_03_A03_H,No-high seal,True
2017-10-10,10/09/2017,NaN,None,NaN,NaN,None,NaN,NaN,"VISp, layer 2/3",NaN,None,NaN,NaN,NaN,NaN,3,3.0,NaN,4-14,09/25/2017,NaN,NaN,NaN,1400,NaN,NaN,no,nucleus_absent,-32.0,-32.0,NaN,NaN,19:56:42,19:55:33,19:56:48,19:56:48,507,NaN,NaN,No,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,H17.03.014.11.04,NaN,NaN,NaN,NaN,NaN,NaN,4.3,NaN,19:42:22,19:44:00,4,dijonh,NaN,NaN,SUCCESS,NaN,Ephys_171009_03_A02_H,No-high seal,True
2017-10-10,10/09/2017,NaN,None,NaN,NaN,None,NaN,NaN,"TCx, layer 3",NaN,None,NaN,NaN,NaN,NaN,2,5.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,5,alexa fill visible post experiment.,NaN,no,nucleus_absent,-60.0,-80.0,Large Bubbles,NaN,20:44:24,20:43:44,20:49:21,20:44:24,059,NaN,NaN,No,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,H17.03.014.11.10,NaN,NaN,NaN,NaN,Unknown Interneuron,NaN,3.7,NaN,20:33:08,20:35:46,7,rustym,NaN,NaN,SUCCESS,NaN,EPhys_171009_03_A03_H,No-high seal,True
2017-10-10,10/09/2017,NaN,None,NaN,NaN,None,NaN,NaN,"TCx, layer 3",NaN,None,NaN,NaN,NaN,NaN,3,4.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,1000,alexa fill visible post experiment.,NaN,no,nucleus_absent,-50.0,-50.0,No Bubbles,NaN,20:13:41,20:12:54,20:16:52,20:13:41,058,NaN,NaN,No,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,H17.03.014.11.10,NaN,NaN,NaN,NaN,Unknown Interneuron,NaN,4.1,NaN,20:02:42,20:04:05,7,rustym,NaN,NaN,SUCCESS,NaN,EPhys_171009_03_A03_H,No-high seal,True


In [16]:
#Mouse
json_df3.head()

,acsfProductionDate,approach.anatomicalLocation,approach.autoRoi,approach.cellHealth,approach.corticalLayer,approach.creCell,approach.depth,approach.detailedLocation,approach.manualRoi,approach.otherPilotName,approach.pilotName,approach.pilotTest01,approach.pilotTest03,approach.pilotTest04,approach.pilotTest05,approach.sliceHealth,attempt,autoRoi,badSweeps,blankFillDate,depth,experimentType,extraction.bfiWitness,extraction.endPipetteR,extraction.extractionNotes,extraction.extractionObservations,extraction.nucleus,extraction.postPatch,extraction.pressureApplied,extraction.retractionPressureApplied,extraction.sampleObservations,extraction.timeEnd,extraction.timeExtractionEnd,extraction.timeExtractionStart,extraction.timeRetractionEnd,extraction.timeRetractionStart,extraction.tubeID,failureCause,failureNotes,flipped,formVersion,freeFailureNotes,internalFillDate,internalSolution.concentrationAlexa,internalSolution.concentrationBiocytin,internalSolution.concentrationRnaseInhibitor,internalSolution.version,internalSolution.volume,limsSpecName,manualRoi,pipetteSpecName,qcNotes,recording.accessR,recording.humanCellTypePrediction,recording.membraneV,recording.pipetteR,recording.rheobase,recording.timeStart,recording.timeWholeCellStart,rigNumber,rigOperator,sliceNotes,sliceQuality,status,successNotes,wellID,post_patch,human?
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,Nucleated Patch - Retraction Pressure,NaN,NaN,NaN,NaN,3,4.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,1000,NaN,Fluorescence in Pipette,no,nucleus_present,-32.0,-32.0,No Bubbles,NaN,15:43:26,15:42:58,15:45:24,15:43:27,055,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Chrna2-Cre_OE25;Ai14-351067.04.01,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,15:37:03,15:38:40,5,rustym,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B03_M,Nuc-high seal,False
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,3,1.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,3000,1.5 min retraction time. Can see alexa fill po...,"Fluorescence in Pipette,Cell Dimmed,Cell Shrunk",no,nucleus_present,-30.0,-30.0,No Bubbles,NaN,15:24:14,15:22:54,15:25:49,15:24:15,356,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Chrna2-Cre_OE25;Ai14-351067.03.02,NaN,NaN,NaN,NaN,NaN,NaN,5.3,NaN,15:14:50,15:16:11,7,lindsayn,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B02_M,Nuc-high seal,False
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 6a",NaN,None,NaN,NaN,NaN,NaN,2,4.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,16.2,NaN,NaN,no,nucleus_absent,-30.0,-30.0,No Bubbles,NaN,15:56:38,15:56:04,15:58:43,15:56:39,406,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Gad2-IRES-Cre;Ai14-350672.04.01,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,15:50:28,15:51:06,6,lisak,NaN,NaN,SUCCESS,Access Resistance Increased,EPhys_171002_01_B01_M,No-high seal,False
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 4",NaN,Nucleated Patch - Retraction Pressure,NaN,NaN,NaN,NaN,3,1.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,10,NaN,Fluorescence in Pipette,no,nucleus_absent,-32.5,-32.5,"Large Bubbles,Solution in Pipette Shank",NaN,14:48:35,14:47:57,14:51:55,14:48:36,053,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Chrna2-Cre_OE25;Ai14-351067.04.01,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,14:41:05,14:42:42,5,rustym,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B03_M,No-high seal,False
2017-10-02,09/29/2017,NaN,None,NaN,NaN,Cre+,NaN,NaN,"VISp, layer 5",NaN,None,NaN,NaN,NaN,NaN,2,2.0,NaN,NaN,09/25/2017,NaN,NaN,NaN,30,No alexa fill post experiment,"Fluorescence in Pipette,Cell Dimmed,Cell Shrunk",no,nucleus_absent,-30.0,-30.0,No Bubbles,NaN,15:56:10,15:55:26,16:01:35,15:56:11,357,NaN,NaN,Yes,2.0.0,NaN,09/29/2017,NaN,NaN,NaN,NaN,NaN,Chrna2-Cre_OE25;Ai14-351067.03.02,NaN,NaN,NaN,NaN,NaN,NaN,5.5,NaN,15:46:38,15:47:49,7,lindsayn,NaN,NaN,SUCCESS,NaN,EPhys_171002_01_B02_M,No-high seal,False


### Create value counts to double check Human and Mouse dataframses

In [17]:
#json_df2['limsSpecName'].value_counts()

In [18]:
#json_df3['limsSpecName'].value_counts()

## HUMAN dataframe

In [19]:
human_df = json_df2[['limsSpecName',
                     'rigOperator',
                     'approach.manualRoi',
                     'manualRoi', 
                     'recording.humanCellTypePrediction',
                     'post_patch',
                     'extraction.tubeID']]

human_df.sort_index(inplace = True)
#approach.manualRoi available data at 2017-10-10 to 2017-10-27
#manualRoi avaible data at 2017-11-27 to Present

In [21]:
human_df.head()

,limsSpecName,rigOperator,approach.manualRoi,manualRoi,recording.humanCellTypePrediction,post_patch,extraction.tubeID
date,,,,,,,
2017-10-10,H17.03.014.11.10,rustym,"TCx, layer 3",NaN,NaN,No-high seal,056
2017-10-10,H17.03.014.11.10,rustym,"TCx, layer 3",NaN,Pyramidal,No-high seal,060
2017-10-10,H17.03.014.11.04,dijonh,"VISp, layer 2/3",NaN,NaN,No-high seal,507
2017-10-10,H17.03.014.11.10,rustym,"TCx, layer 3",NaN,Unknown Interneuron,No-high seal,059
2017-10-10,H17.03.014.11.10,rustym,"TCx, layer 3",NaN,Unknown Interneuron,No-high seal,058


In [22]:
human_df.tail()

,limsSpecName,rigOperator,approach.manualRoi,manualRoi,recording.humanCellTypePrediction,post_patch,extraction.tubeID
date,,,,,,,
2018-05-22,H18.03.012.11.13.02,dijonh,NaN,TCx3,Pyramidal,No-low seal,PBS4_180522_505_A01
2018-05-22,H18.03.012.11.13.02,dijonh,NaN,TCx2,FS Interneuron,No-low seal,PBS4_180522_506_A01
2018-05-22,H18.03.012.11.11.01,lindsayn,NaN,TCx2,Pyramidal,No-high seal,P8S4_180522_353_A01
2018-05-22,H18.03.012.11.13.02,dijonh,NaN,TCx3,FS Interneuron,No-low seal,PBS4_180522_507_A01
2018-05-22,H18.03.012.11.13.02,dijonh,NaN,TCx3,NaN,No-low seal,PBS4_180522_508_A01


In [23]:
human_df['post_patch'].value_counts()

No-low seal      123
No-high seal     108
Nuc-low seal      49
Nuc-high seal     36
Entire cell        3
Name: post_patch, dtype: int64

## MOUSE dataframe

In [24]:
mouse_df = json_df3[['limsSpecName',
                     'rigOperator',
                     'approach.manualRoi',
                     'manualRoi', 
                     'post_patch',
                     'extraction.tubeID']]

mouse_df.sort_index(inplace = True)
#approach.manualRoi available data at 2017-10-02 to 2017-11-08
#manualRoi avaible data at 2017-11-08 to Present

In [25]:
mouse_df.head()

,limsSpecName,rigOperator,approach.manualRoi,manualRoi,post_patch,extraction.tubeID
date,,,,,,
2017-10-02,Chrna2-Cre_OE25;Ai14-351067.04.01,rustym,"VISp, layer 5",NaN,Nuc-high seal,055
2017-10-02,Chrna2-Cre_OE25;Ai14-351067.03.02,lindsayn,"VISp, layer 5",NaN,Nuc-high seal,356
2017-10-02,Gad2-IRES-Cre;Ai14-350672.04.01,lisak,"VISp, layer 6a",NaN,No-high seal,406
2017-10-02,Chrna2-Cre_OE25;Ai14-351067.04.01,rustym,"VISp, layer 4",NaN,No-high seal,053
2017-10-02,Chrna2-Cre_OE25;Ai14-351067.03.02,lindsayn,"VISp, layer 5",NaN,No-high seal,357


In [26]:
mouse_df.tail()

,limsSpecName,rigOperator,approach.manualRoi,manualRoi,post_patch,extraction.tubeID
date,,,,,,
2018-05-24,Nos1-CreERT2;Sst-IRES-FlpO;Ai65-391574.03.02,lindsayn,NaN,VISp6a,Nuc-high seal,P8S4_180524_351_A01
2018-05-24,Nos1-CreERT2;Sst-IRES-FlpO;Ai65-391574.03.01,dijonh,NaN,VISp4,Nuc-high seal,PBS4_180524_502_A01
2018-05-24,Htr3a-Cre_NO152;Ai14-390983.03.01,lisak,NaN,VISp4,No-high seal,P9S4_180524_404_A01
2018-05-24,Htr3a-Cre_NO152;Ai14-390983.03.01,lisak,NaN,VISp6a,No-high seal,P9S4_180524_405_A01
2018-05-24,Nos1-CreERT2;Sst-IRES-FlpO;Ai65-391574.03.01,dijonh,NaN,VISp6a,No-low seal,PBS4_180524_501_A01


In [27]:
mouse_df['post_patch'].value_counts()

No-high seal     527
No-low seal      434
Nuc-high seal    312
Nuc-low seal     282
Entire cell       40
Name: post_patch, dtype: int64